In [41]:
import os
import pandas as pd
from core import read_dom_table

In [70]:
padloc_hmm_info = pd.read_table('/home/gridsan/pdeweirdt/.conda/envs/padloc/data/hmm_meta.txt')

## Set up hmm databases

In [4]:
padloc_db = '/home/gridsan/pdeweirdt/.conda/envs/padloc/data/hmm/padlocdb.hmm'
df_db = '/home/gridsan/pdeweirdt/Downloads/defense_finder_db.hmm'

In [5]:
if 'defense_finder_db.hmm' not in os.listdir('/home/gridsan/pdeweirdt/Downloads/'):
    os.system('cat /home/gridsan/pdeweirdt/Downloads/defense-finder-models/profiles/*.hmm > ' + df_db)

## Search Refseq500

In [8]:
refseq_500_novel_df = pd.read_csv('../data/processed/refseq_500_novel_defense.csv')
len(refseq_500_novel_df)

3376

In [13]:
refseq_500_novel_ids = '../data/interim/refseq_500_novel_ids.txt'

In [28]:
refseq_500_novel_df[['product_accession']].to_csv(refseq_500_novel_ids, header=False, index=False)

In [33]:
refseq_500_seqs = '../data/interim/refseq_500_unique_seqs.faa'

In [34]:
refseq_500_novel_seqs = '../data/interim/refseq_500_unique_novel_seqs.faa'

In [35]:
!conda run -n beaker seqtk subseq {refseq_500_seqs} {refseq_500_novel_ids} > {refseq_500_novel_seqs}

In [36]:
refseq_500_padloc_out = '../data/interim/refeq_500_padloc_domtbl.txt'
refseq_500_padloc_temp_out = '../data/tmp/refeq_500_padloc_tmp.txt'

In [39]:
!conda run -n beaker hmmsearch --cpu 40 --acc --noali --domtblout {refseq_500_padloc_out} -o {refseq_500_padloc_temp_out} {padloc_db} {refseq_500_novel_seqs}

In [40]:
refseq_500_df_out = '../data/interim/refseq_500_df_domtbl.txt'
refseq_500_df_temp_out = '../data/tmp/refeq_500_df_tmp.txt'

In [51]:
!conda run -n beaker hmmsearch --cpu 40 --acc --noali --domtblout {refseq_500_df_out} -o {refseq_500_df_temp_out} {df_db} {refseq_500_novel_seqs}


## Seach E. coli 3000

In [9]:
ecoli_3000_novel_df = pd.read_csv('../data/ecoli/processed/unique_novel_defense.csv')
len(ecoli_3000_novel_df)

1374

In [17]:
ecoli_3000_novel_ids = '../data/ecoli/interim/ecoli_novel_ids.txt'

In [29]:
ecoli_3000_novel_df[['product_accession']].to_csv(ecoli_3000_novel_ids, header=False, index=False)

In [30]:
ecoli_3000_seqs = '../data/ecoli/interim/unique_seqs.faa'

In [31]:
ecoli_3000_novel_seqs = '../data/ecoli/interim/unique_novel_seqs.faa'

In [32]:
!conda run -n beaker seqtk subseq {ecoli_3000_seqs} {ecoli_3000_novel_ids} > {ecoli_3000_novel_seqs}

In [56]:
ecoli_3k_padloc_out = '../data/ecoli/interim/ecoli_3k_padloc_domtbl.txt'
ecoli_3k_padloc_temp_out = '../data/tmp/ecoli_3k_padloc_tmp.txt'

In [57]:
!conda run -n beaker hmmsearch --cpu 40 --acc --noali --domtblout {ecoli_3k_padloc_out} -o {ecoli_3k_padloc_temp_out} {padloc_db} {ecoli_3000_novel_seqs}


In [58]:
ecoli_3k_df_out = '../data/interim/ecoli_3k_df_domtbl.txt'
ecoli_3k_df_temp_out = '../data/tmp/ecol_3k_df_tmp.txt'

In [59]:
!conda run -n beaker hmmsearch --cpu 40 --acc --noali --domtblout {ecoli_3k_df_out} -o {ecoli_3k_df_temp_out} {df_db} {ecoli_3000_novel_seqs}


## Analyze hits

In [64]:
refseq_500_padloc_domtbl = read_dom_table(refseq_500_padloc_out)
refseq_500_padloc_domtbl['hmm_db'] = 'padloc'
refseq_500_padloc_domtbl['seq_db'] = 'refseq_500'
refseq_500_df_domtbl = read_dom_table(refseq_500_df_out)
refseq_500_df_domtbl['hmm_db'] = 'df'
refseq_500_df_domtbl['seq_db'] = 'refseq_500'
ecoli_3k_padloc_domtbl = read_dom_table(ecoli_3k_padloc_out)
ecoli_3k_padloc_domtbl['hmm_db'] = 'padloc'
ecoli_3k_padloc_domtbl['seq_db'] = 'ecoli_3k'
ecoli_3k_df_domtbl = read_dom_table(ecoli_3k_df_out)
ecoli_3k_df_domtbl['hmm_db'] = 'df'
ecoli_3k_df_domtbl['seq_db'] = 'ecoli_3k'

In [68]:
for dom_df in [refseq_500_padloc_domtbl, refseq_500_df_domtbl, ecoli_3k_padloc_domtbl, ecoli_3k_df_domtbl]:
    dom_df['hmm_ali_len'] = dom_df['hmm_to'] - dom_df['hmm_from'] + 1
    dom_df['env_ali_len'] = dom_df['env_to'] - dom_df['env_from'] + 1
    dom_df['hmm_cov'] = dom_df['hmm_ali_len'] / dom_df['qlen']
    dom_df['t_cov'] = dom_df['env_ali_len'] / dom_df['tlen']


In [75]:
merged_refseq_500_padloc_domtbl = (refseq_500_padloc_domtbl.merge(padloc_hmm_info[['hmm.accession', 'e.val.threshold', 
                                                                                   'hmm.coverage.threshold', 'target.coverage.threshold']]
                                                                  .rename(columns={'hmm.accession': 'query_accession'}), 
                                                                  how='inner', 
                                                                  on='query_accession'))

In [84]:
filtered_refseq_500_padloc_domtbl = merged_refseq_500_padloc_domtbl[(merged_refseq_500_padloc_domtbl['dom_i_evalue'] < merged_refseq_500_padloc_domtbl['e.val.threshold']) & 
                                                                    (merged_refseq_500_padloc_domtbl['hmm_cov'] > merged_refseq_500_padloc_domtbl['hmm.coverage.threshold']) &
                                                                    (merged_refseq_500_padloc_domtbl['t_cov'] > merged_refseq_500_padloc_domtbl['target.coverage.threshold']) & 
                                                                    (~merged_refseq_500_padloc_domtbl['query'].str.startswith('PDC'))]
filtered_refseq_500_padloc_domtbl['target'].nunique()

88

In [85]:
merged_ecoli_3k_padloc_domtbl = (ecoli_3k_padloc_domtbl.merge(padloc_hmm_info[['hmm.accession', 'e.val.threshold', 
                                                                                   'hmm.coverage.threshold', 'target.coverage.threshold']]
                                                                  .rename(columns={'hmm.accession': 'query_accession'}), 
                                                                  how='inner', 
                                                                  on='query_accession'))

In [86]:
filtered_ecoli_3k_padloc_domtbl = merged_ecoli_3k_padloc_domtbl[(merged_ecoli_3k_padloc_domtbl['dom_i_evalue'] < merged_ecoli_3k_padloc_domtbl['e.val.threshold']) & 
                                                                    (merged_ecoli_3k_padloc_domtbl['hmm_cov'] > merged_ecoli_3k_padloc_domtbl['hmm.coverage.threshold']) &
                                                                    (merged_ecoli_3k_padloc_domtbl['t_cov'] > merged_ecoli_3k_padloc_domtbl['target.coverage.threshold']) & 
                                                                    (~merged_ecoli_3k_padloc_domtbl['query'].str.startswith('PDC'))]
filtered_ecoli_3k_padloc_domtbl['target'].nunique()

48

In [92]:
(filtered_ecoli_3k_padloc_domtbl.sort_values('dom_i_evalue')
 .groupby('query')
 .head(1))

,target,target_accession,tlen,query,query_accession,qlen,seq_evalue,seq_score,seq_bias,dom_n,dom_of,dom_c_evalue,dom_i_evalue,dom_score,dom_bias,hmm_from,hmm_to,ali_from,ali_to,env_from,env_to,acc,targ_description,hmm_db,seq_db,hmm_ali_len,env_ali_len,hmm_cov,t_cov,e.val.threshold,hmm.coverage.threshold,target.coverage.threshold
3226,WP_160191500.1,-,906.0,REase_III_00008,PDLC03654,1154.0,0.000000e+00,1835.7,12.0,1.0,1.0,0.000000e+00,0.000000e+00,1835.4,12.0,233.0,1128.0,9.0,904.0,4.0,906.0,1.00,-,padloc,ecoli_3k,896.0,903.0,0.776430,0.996689,1.000000e-05,0.5,0.3
1478,WP_251324570.1,-,622.0,HEC-05_WP_156276309.1,PDLC04151,600.0,6.000000e-231,764.5,18.1,1.0,1.0,1.100000e-233,7.400000e-231,764.2,18.1,20.0,594.0,2.0,601.0,1.0,610.0,0.96,-,padloc,ecoli_3k,575.0,610.0,0.958333,0.980707,1.000000e-20,0.7,0.7
1813,WP_196354812.1,-,366.0,HEC-08_WP_093018341.1,PDLC04162,358.0,3.300000e-174,575.2,7.0,1.0,1.0,1.100000e-176,3.600000e-174,575.1,7.0,6.0,357.0,11.0,365.0,1.0,366.0,0.98,-,padloc,ecoli_3k,352.0,366.0,0.983240,1.000000,1.000000e-20,0.7,0.7
3003,WP_236323677.1,-,237.0,REase_II_00104,PDLC03502,301.0,4.900000e-158,521.2,3.0,1.0,1.0,4.000000e-161,5.500000e-158,521.0,3.0,1.0,237.0,1.0,237.0,1.0,237.0,1.00,-,padloc,ecoli_3k,237.0,237.0,0.787375,1.000000,1.000000e-05,0.5,0.3
2284,WP_064764923.1,-,390.0,pAgo-associated_WP_077117439.1,PDLC02404,368.0,1.000000e-132,439.5,0.0,1.0,1.0,5.000000e-135,1.200000e-132,439.4,0.0,2.0,367.0,8.0,384.0,7.0,385.0,0.99,-,padloc,ecoli_3k,366.0,379.0,0.994565,0.971795,1.000000e-08,0.5,0.5
2629,WP_251324570.1,-,622.0,HEC-05_WP_096029243.1,PDLC04150,583.0,1.500000e-130,433.0,22.4,1.0,1.0,2.800000e-133,1.900000e-130,432.6,22.4,1.0,579.0,2.0,598.0,2.0,601.0,0.86,-,padloc,ecoli_3k,579.0,600.0,0.993139,0.964630,1.000000e-20,0.7,0.7
1288,WP_064764923.1,-,390.0,Aga_Helical-REase,PDLC02411,378.0,6.000000e-107,354.2,0.0,1.0,1.0,3.000000e-109,6.800000e-107,354.0,0.0,2.0,377.0,7.0,382.0,6.0,383.0,0.98,-,padloc,ecoli_3k,376.0,378.0,0.994709,0.969231,1.000000e-08,0.5,0.5
807,WP_021529611.1,-,253.0,Tin,PDLC02359,239.0,2.400000e-98,324.5,1.5,1.0,1.0,4.000000e-101,2.700000e-98,324.3,1.5,1.0,239.0,2.0,249.0,2.0,249.0,0.99,-,padloc,ecoli_3k,239.0,248.0,1.000000,0.980237,1.000000e-20,0.8,0.8
2875,WP_032142271.1,-,333.0,REase_II_00053,PDLC03451,293.0,3.400000e-86,285.1,0.8,1.0,1.0,3.000000e-89,4.100000e-86,284.9,0.8,24.0,291.0,63.0,332.0,49.0,333.0,0.91,-,padloc,ecoli_3k,268.0,285.0,0.914676,0.855856,1.000000e-05,0.5,0.3
2974,WP_337051981.1,-,116.0,REase_II_00070,PDLC03468,214.0,4.500000e-74,244.7,0.1,1.0,1.0,3.600000e-77,5.000000e-74,244.6,0.1,99.0,214.0,1.0,116.0,1.0,116.0,0.99,-,padloc,ecoli_3k,116.0,116.0,0.542056,1.000000,1.000000e-05,0.5,0.3


In [88]:
df_i_eval = 0.001
df_hmm_cov = 0.4

In [90]:
filtered_reqseq_500_df_domtbl = refseq_500_df_domtbl[(refseq_500_df_domtbl['dom_i_evalue'] < df_i_eval) & 
                                                     (refseq_500_df_domtbl['hmm_cov'] > df_hmm_cov)]
filtered_reqseq_500_df_domtbl['target'].nunique()

65

In [91]:
filtered_ecoli_3k_df_domtbl = ecoli_3k_df_domtbl[(ecoli_3k_df_domtbl['dom_i_evalue'] < df_i_eval) & 
                                                     (ecoli_3k_df_domtbl['hmm_cov'] > df_hmm_cov)]
filtered_ecoli_3k_df_domtbl['target'].nunique()

56